In [ ]:
from random import choices, randrange, choice
from collections import defaultdict
import igraph as ig

In [ ]:
def postprocess(g):
    alone_nodes = g.vs(_degree_eq=0)
    if alone_nodes:
        for alone_node in alone_nodes:
            while True:
                random_edge = choice(g.es)
                source = random_edge.source
                target = random_edge.target
                if g.vs[source].degree()>2 and g.vs[target].degree()>2:
                    g.delete_edges(random_edge)
                    g.add_edge(alone_node.index, target)
                    break
    return g

In [ ]:
g = ig.load('../real/airports.net') # our graph
print(g.summary())
degree = g.degree()
print(sum(degree)/len(degree))
print()

g.delete_vertices(v.index for _, v in sorted(zip(g.degree(), g.vs))[:-500])
ecount = g.ecount()
g.delete_edges(sample(range(ecount), ecount-1000))
g = postprocess(g)

print(g.summary())
degree = g.degree()
print(sum(degree)/len(degree))
print(len(g.vs(_degree_eq=0)))

In [ ]:
def WS(n):
    g = ig.Graph(n)
    sources = list(range(n))
    for i in (1, 2):
        targets = sources[i:] + sources[0:i]
        g.add_edges(list(zip(sources, targets)))
    return g

g = WS(500)
print(g.summary())
degree = g.degree()
print(sum(degree)/len(degree))
print(len(g.vs(_degree_eq=0)))

In [ ]:
def BA(n, m, m0):
    vertices = list(range(m0))
    forbiddenEdges = set()
    degrees = [m0-1] * m0    
    edges = []
    
    g = ig.Graph(n)
    for i in range(m0):
        for j in range(i+1, m0):
            edges.append((i, j))
    g.add_edges(edges)
    edges = []
    for index in range(m0, n):
        forbiddenEdges.clear()      
        count = 0
        while count < m:
            target = choices(vertices, degrees)[0]
            if target not in forbiddenEdges:
                edges.append((index, target))
                forbiddenEdges.add(target)
                degrees[target] += 1                
                count += 1
        vertices.append(index)
        degrees.append(m)

    g.add_edges(edges)
    return g

#g = BA(3618, 3, 85)
#g = BA(3618, 4, 4)
g = BA(500, 2, 5)
print(g.summary())
degree = g.degree()
print(sum(degree)/len(degree))
print(len(g.vs(_degree_eq=0)))
#14142

In [ ]:
def GNM(n, m):
    forbiddenEdges = defaultdict(set)
    rangeSet = set(range(n))
    edges = []
    for _ in range(m):
        i = randrange(n)
        j = choice(tuple(
            rangeSet.difference(forbiddenEdges[i].union({i}))
        ))
        forbiddenEdges[i].add(j)
        forbiddenEdges[j].add(i)
        edges.append((i,j))
    g = ig.Graph(n) 
    g.add_edges(edges)
    return g

g = GNM(3618, 14142)
#g = GNM(500, 1000)
g = postprocess(g)
print(g.summary())
degree = g.degree()
print(sum(degree)/len(degree))
print(len(g.vs(_degree_eq=0)))